<a href="https://colab.research.google.com/github/Ronnypetson/titanic/blob/master/MNIST_Maromba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Experimentos do Produto Interno Maromba no MNIST

### Ambiente

In [1]:
import torch
import numpy as np
import torch.nn as nn
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
from torchvision import transforms
from torchvision.transforms import ToTensor, Normalize
from torch.optim import Adam, SGD

import pandas as pd
from sklearn.metrics import accuracy_score
# !pip3 install ipympl
# !pip3 install mpl_interactions
# %matplotlib widget

import matplotlib.pylab as plt
import plotly.express as px
# import mpl_interactions.ipyplot as iplt
import time
from IPython import display
from IPython.core.debugger import Pdb

device = "cuda:0" if torch.cuda.is_available() else "cpu"

# %matplotlib inline
# from google.colab import output
# output.enable_custom_widget_manager()

### Dataset

In [2]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
    ]
)

tr = ToTensor()
# cifar10_norm = Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
cifar10_norm = Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

channels = 1
img_dim = 28

def _transform(x):
  # x = x.resize((channels, img_dim, img_dim))
  # return cifar10_norm(tr(x)).reshape(-1)
  # return (tr(x) / 255.0).reshape(-1)
  # return transform(x).reshape(-1)
  return (tr(x) * 2.0 - 1.0).reshape(-1)
  # return (tr(x).mean(dim=0)).reshape(-1)

bsize = 32 ###

# SOURCE_DATASET = MNIST
SOURCE_DATASET = FashionMNIST
# SOURCE_DATASET = CIFAR10

MNIST_train_data = SOURCE_DATASET(
    "MNIST_root/",
    download=True,
    train=True,
    transform=_transform,
)
train_data_loader = torch.utils.data.DataLoader(
    MNIST_train_data,
    batch_size=bsize,
    shuffle=True,
    num_workers=1,
)

MNIST_test_data = SOURCE_DATASET(
    "MNIST_root_test/",
    download=True,
    train=False,
    transform=_transform,
)
test_data_loader = torch.utils.data.DataLoader(
    MNIST_test_data,
    batch_size=bsize,
    shuffle=True,
    num_workers=1,
)

100%|██████████| 26421880/26421880 [00:02<00:00, 11549026.41it/s]


Extracting MNIST_root/FashionMNIST/raw/train-images-idx3-ubyte.gz to MNIST_root/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 209867.01it/s]


Extracting MNIST_root/FashionMNIST/raw/train-labels-idx1-ubyte.gz to MNIST_root/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3944280.56it/s]


...

Extracting MNIST_root_test/FashionMNIST/raw/train-labels-idx1-ubyte.gz to MNIST_root_test/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3870084.68it/s]


Extracting MNIST_root_test/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_root_test/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 24536678.40it/s]

Extracting MNIST_root_test/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_root_test/FashionMNIST/raw



In [3]:
# import torch.nn.functional as F

# # config = {
# #     "features": {
# #         "pre-set": channels,
# #         "sets":    [1024,    1024,    1,],
# #         "samples": [25,      1,       1024,],
# #     },
# #     "params": {
# #         "sets":    [4,       1,       num_classes,],
# #         "samples": [(76, 1), (1, 5),  (1025, 1),],
# #         "is conv": [True,    True,    False,]
# #     },
# # }

# class Net(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(3, 4, 5, padding="same", bias=False)
#         # self.pool = nn.MaxPool2d(2, 2)
#         # self.conv2 = nn.Conv2d(6, 8, 5, padding="same", bias=False)
#         self.conv2 = nn.Conv2d(4, 1, 1, padding="same", bias=False)
#         self.fc1 = nn.Linear(1024, 10, bias=False)
#         # self.fc2 = nn.Linear(100, 10, bias=False)

#     def forward(self, x):
#         x = F.elu(self.conv1(x))
#         x = F.elu(self.conv2(x))
#         # x = F.elu(self.conv3(x))
#         x = torch.flatten(x, 1) # flatten all dimensions except batch
#         x = self.fc1(x)
#         return x

# net = Net()

In [4]:
# import torch.optim as optim
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [5]:
# trainloader = train_data_loader
# for epoch in range(2):  # loop over the dataset multiple times
#     running_loss = 0.0
#     for i, data in enumerate(trainloader, 0):
#         # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data
#         # zero the parameter gradients
#         optimizer.zero_grad()
#         # forward + backward + optimize
#         outputs = net(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         # print statistics
#         running_loss += loss.item()
#         if i % 60 == 59:    # print every 60 mini-batches
#             print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 60:.3f}')
#             running_loss = 0.0
# print('Finished Training')

In [6]:
# img, lbl = next(iter(test_data_loader))
# img = img.reshape(-1, 3, 32, 32)
# img = img.permute(0, 2, 3, 1).cpu().detach().numpy()

In [7]:
# img[0].min(), img[0].max(), img[0].mean()

# cifar10_classes = [
#     "airplane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"
# ]
# _idx = 3
# print(cifar10_classes[lbl[_idx].item()])
# plt.imshow(img[_idx])

In [8]:
def _cat2d(rows, cols, d=32):
  """
  Index in the log-softmax scale.
  After sotmax (in the partition dimension)
  -inf --> 0
  1.0  --> 1
  """
  assert rows + cols <= d
  inf = 1.0
  idx = np.zeros((rows, cols, d)) - inf
  for row in range(rows):
    for col in range(cols):
      idx[row, col, row] = 1.0
      idx[row, col, rows + col] = 1.0
  idx = torch.from_numpy(idx)
  idx = idx.reshape(rows * cols, d)
  return idx

def cartesian_idx(rows, cols, chs=1, d=2, offset=0):
  idx = np.zeros((rows, cols, chs, d))
  for row in range(rows):
    for col in range(cols):
      for ch in range(chs):
        idx[row, col, ch, 0] = 2.0 * ((row + offset) / rows) - 1.0
        idx[row, col, ch, 1] = 2.0 * ((col + offset) / cols) - 1.0
        idx[row, col, ch, 2] = 2.0 * ((ch  + offset) /  chs) - 1.0
  idx = torch.from_numpy(idx)
  idx = idx.reshape(rows * cols * chs, d)
  return idx

### Kernels, similaridades e funções de índice

#### Kernels

In [9]:
def _soft_kernel(idxu, part_dim):
  """
  idxu: M x d_u x d_idx
  """
  m, d_u, d_idx = idxu.shape
  assert d_idx % part_dim == 0
  range = 20.0
  norm_idxu = range * idxu.reshape(m, d_u, -1, part_dim) - (range / 2.0)
  norm_idxu = torch.softmax(norm_idxu, dim=-1)
  dim_norm = (d_idx // part_dim) ** 0.5
  norm_idxu = norm_idxu.reshape(m, d_u, d_idx) / dim_norm
  return norm_idxu

def _cosine_kernel(idxu, *args, **kwargs):
  """
  idxu: M x d_u x d_idx
  """
  # TODO: compute min_idx, max_idx and normalize
  m, d_u, d_idx = idxu.shape
  min_idxu = torch.min(idxu, dim=1)[0].unsqueeze(1)
  max_idxu = torch.max(idxu, dim=1)[0].unsqueeze(1)
  eps = 1e-4
  idxu = (idxu - min_idxu) / (max_idxu - min_idxu + eps)
  norm_idxu = idxu / (torch.norm(idxu, dim=-1).unsqueeze(-1) + eps)
  # Reverse kernel trick for polynomial x^2
  idxu2 = norm_idxu.reshape(-1, d_idx, 1)
  idxu2 = torch.bmm(idxu2, idxu2.permute(0, 2, 1)).reshape(m, d_u, -1)
  # return norm_idxu
  return idxu2

#### Similaridades

In [10]:
def squared_cosine(idxu, idxv):
  """
  idxu: pre_shape x d_idx
  idxv: pre_shape x d_idx
  """
  assert idxu.shape == idxv.shape
  d_idx = idxu.shape[-1]
  sim = torch.bmm(
      idxu.reshape(-1, 1, d_idx),
      idxv.reshape(-1, d_idx, 1),
  )
  # sim = (torch.exp(sim) - 1.0) / (1.718)
  sim = sim ** 4.0
  return sim

def relu_cosine(idxu, idxv, bias=0.9):
  """
  idxu: pre_shape x d_idx
  idxv: pre_shape x d_idx
  """
  assert idxu.shape == idxv.shape
  d_idx = idxu.shape[-1]
  sim = nn.functional.relu(
      torch.bmm(
          idxu.reshape(-1, 1, d_idx),
          idxv.reshape(-1, d_idx, 1),
      )
      - bias
  )
  sim = sim.reshape(idxu.shape[:-1])
  return sim

#### Funções-valor

In [11]:
def vecsum(u, v):
  return u + v

def vecmean(u, v):
  return (u + v) / 2.0

def vecprod(u, v):
  """
  Element-wise product. NOT dot product.
  """
  return u * v

#### Dots

##### Utilities

In [12]:
class Pairwise:
  def __init__(self, f):
    """
    f: (pre_shape x d_val, pre_shape x d_val) -> pre_shape x d_val_out
    """
    self._f = f

  def __call__(self, u, v):
    """
    u: pre_shape_u x d_u x d_val
    v: pre_shape_v x d_v x d_val
    ans: pre_shape_u x pre_shape_v x d_u x d_v x d_val_out
    """
    ps_u, ps_v = u.shape[:-2], v.shape[:-2]
    pps_u, pps_v = np.prod(ps_u), np.prod(ps_v)
    d_u, d_val = u.shape[-2:]
    d_v, d_valv = v.shape[-2:]
    assert d_val == d_valv
    # u, v: pps_u x pps_v x d_u x d_v x d_val
    u = u.reshape(pps_u,     1, d_u,   1, d_val)
    v = v.reshape(    1, pps_v,   1, d_v, d_val)
    u = u.repeat(     1, pps_v,   1, d_v,     1)
    v = v.repeat( pps_u,     1, d_u,   1,     1)
    # fuv: ps_u x ps_v x d_u x d_v x d_val_out
    fuv = self._f(u, v)
    fuv = fuv.reshape(*ps_u, *ps_v, d_u, d_v, -1)
    return fuv

def __minmax_normalize(idxu):
  """
  idxu: pre_shape x d_idx
  """
  min_idxu = torch.min(idxu, dim=1)[0].unsqueeze(1)
  max_idxu = torch.max(idxu, dim=1)[0].unsqueeze(1)
  eps = 1e-6
  idxu = min_idxu + ((idxu - min_idxu) / (max_idxu - min_idxu + eps))
  return idxu

def minmax_normalize(idxu):
  """
  idxu: pre_shape x d_idx
  """
  min_idxu = torch.min(idxu, dim=1)[0].unsqueeze(1)
  max_idxu = torch.max(idxu, dim=1)[0].unsqueeze(1)
  eps = 1e-6
  idxu = 2.0 * ((idxu - min_idxu) / (max_idxu - min_idxu + eps)) - 1.0
  return idxu

def norm_normalize(u):
  """
  u: pre_shape x d_val
  """
  eps = 1e-6
  u = u / (u.norm(dim=-1).unsqueeze(-1) + eps)
  return u

def normalized(idxu):
  """
  idxu: pre_shape x d_idx
  """
  idxu = minmax_normalize(idxu)
  idxu = norm_normalize(idxu)
  return idxu

from functools import lru_cache

@lru_cache()
def get_eye(m, d_u, d_v, n, device="cpu"):
  eye = (
      torch.eye(max(d_u, d_v))
      [:d_u, :d_v]
      .unsqueeze(0)
      .unsqueeze(-1)
      .repeat(m, 1, 1, n)
      .to(device)
  )
  return eye

def log_sinkhorn(log_alpha, n_iter):
    """ https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/DL2/sampling/permutations.html
    [1] Sinkhorn, Richard and Knopp, Paul.
    Concerning nonnegative matrices and doubly stochastic
    matrices. Pacific Journal of Mathematics, 1967
    Args:
      log_alpha: 2D tensor (a matrix of shape [N, N])
        or 3D tensor (a batch of matrices of shape = [batch_size, N, N])
      n_iters: number of sinkhorn iterations
    """
    for _ in range(n_iter):
        # log_alpha = log_alpha - torch.logsumexp(log_alpha, -1, keepdim=True)
        # log_alpha = log_alpha - torch.logsumexp(log_alpha, -2, keepdim=True)
        log_alpha = log_alpha - torch.logsumexp(log_alpha, -2, keepdim=True)
        log_alpha = log_alpha - torch.logsumexp(log_alpha, -1, keepdim=True)
    return log_alpha.exp()

##### Dot products

In [32]:
def _rdot(u, v, *args):
  """
  "Regular Dot product"
  u: M x d_u x d_val
  v: N x d_v x d_val
  """
  m, d_u, d_val = u.shape
  n, d_v, _d_val = v.shape
  if d_u != d_v:
    return _nsbmd(u, v, *args)
  assert _d_val == d_val
  dot = (
      u.permute(0, 2, 1).reshape(-1, d_u)
      @ v.permute(1, 0, 2).reshape(d_v, -1)
  ).reshape(m, d_val, n, d_val).permute(0, 2, 1, 3)
  dot = torch.diagonal(dot, dim1=2, dim2=3)
  return dot

def _nsbmd(u, v, idxu, idxv, bias=0.5) -> torch.Tensor:
  """
  "Non-linear Similarity Batch Maromba Dot"
  u: M x d_u x d_val
  v: N x d_v x d_val
  idxu: M x d_u x d_idx
  idxv: N x d_v x d_idx
  """
  Pdb().set_trace()
  m, d_u, d_idx  = idxu.shape
  n, d_v, d_idxv = idxv.shape
  d_val = u.shape[-1]
  assert d_idx == d_idxv
  assert d_val == v.shape[-1]
  assert (m, d_u) == u.shape[:2]
  assert (n, d_v) == v.shape[:2]
  idxu = normalized(idxu)
  idxv = normalized(idxv)
  # idxuv: M x d_u x d_v x N
  ###
  # idxuv = idxu.reshape(m * d_u, d_idx) @ idxv.reshape(n * d_v, d_idx).T
  # idxuv = idxuv.reshape(m, d_u, n, d_v).permute(0, 1, 3, 2)
  # mag = 80.0
  # # idxuv = nn.functional.softmax(mag * idxuv - (mag / 2.0), dim=2)
  # # idxuv = nn.functional.softmax(mag * idxuv, dim=2)
  # idxuv = (
  #     log_sinkhorn(mag * idxuv.permute(0, 3, 1, 2), 6)
  #     .permute(0, 2, 3, 1)
  # )
  ###
  from random import randint
  bsidx = randint(0, m - 1)
  m_ = 1 # m,
  idxuv = (
      idxu[bsidx].reshape(m_ * d_u, d_idx)
      @ idxv.reshape(n * d_v, d_idx).T
  )
  idxuv = idxuv.reshape(m_, d_u, n, d_v).permute(0, 1, 3, 2)
  mag = 80.0
  # siter = int(np.log((d_u + d_v) // 2)) * 6
  siter = 6
  idxuv = (
      log_sinkhorn(mag * idxuv.permute(0, 3, 1, 2), siter)
      .permute(0, 2, 3, 1)
  )
  idxuv = idxuv.repeat(m, 1, 1, 1)
  ###
  # from random import randint
  # bsidx = randint(0, m - 1)
  # m_ = 1
  # mag = 80.0
  # # idxu = log_sinkhorn(mag * idxu[bsidx].unsqueeze(0), 6)
  # # idxv = log_sinkhorn(mag * idxv, 6)
  # idxu = nn.functional.softmax(mag * idxu[bsidx].unsqueeze(0), dim=1)
  # idxv = nn.functional.softmax(mag * idxv, dim=1)
  # idxuv = idxu.reshape(m_ * d_u, d_idx) @ idxv.reshape(n * d_v, d_idx).T
  # idxuv = idxuv.reshape(m_, d_u, n, d_v).permute(0, 1, 3, 2)
  # idxuv = idxuv.repeat(m, 1, 1, 1)
  ###
  # uidxuv: (M x d_val x d_v x N) -> (N x d_v x d_val x M)
  uidxuv = (
      torch.bmm(
        u.permute(0, 2, 1),
        idxuv.reshape(m, d_u, d_v * n)
      )
      .reshape(m, d_val, d_v, n)
      .permute(3, 2, 1, 0)
  )
  # uidxuvv: N x M x d_val x d_val
  uidxuvv = (
      torch.bmm(
          uidxuv.permute(0, 3, 2, 1).reshape(n * m, d_val, d_v),
          v.unsqueeze(1).repeat(1, m, 1, 1).reshape(n * m, d_v, d_val)
      )
      .reshape(n, m, d_val, d_val)
  )
  # dot: M x N x d_val
  dot = torch.diagonal(uidxuvv, dim1=2, dim2=3)
  dot = dot.permute(1, 0, 2)
  return dot

#### Índices

In [14]:
def _fast_kernel_idx_sum(idxu, idxv, k, idx_part):
  """
  k: callable: A x B x C -> A x B x C
  idxu: M x d_u x d_idx
  idxv: N x d_v x d_idx
  """
  ### idxu MUST be the input mini-batch
  batch_m = 1 # idxu.shape[0]
  # idxu = idxu.mean(dim=0).unsqueeze(0)
  ###
  m, d_u, d_idx = idxu.shape
  n, d_v, _ = idxv.shape
  assert d_idx == idxv.shape[-1]
  # kidxu: M x d_u x d_idx
  # kidxv: N x d_v x d_idx
  kidxu = k(idxu, idx_part)
  kidxv = k(idxv, idx_part)
  d_idx_k = kidxu.shape[-1]
  assert kidxu.shape[:-1] == idxu.shape[:-1]
  assert kidxv.shape[:-1] == idxv.shape[:-1]
  # kiTi: (M * d_idx) x d_idx(k)
  # kjTj: (N * d_idx) x d_idx(k)
  iTki = torch.bmm(idxu.permute(0, 2, 1), kidxu).reshape(m * d_idx, d_idx_k)
  jTkj = torch.bmm(idxv.permute(0, 2, 1), kidxv).reshape(n * d_idx, d_idx_k)
  ski = kidxu.sum(dim=1)
  skj = kidxv.sum(dim=1)
  norm = (ski @ skj.T).unsqueeze(-1)
  # sidx: (M * d_idx) x N + (N * d_idx) x M
  sidx = (
      (iTki @ skj.T).reshape(m, d_idx, n).permute(0, 2, 1)
      + (jTkj @ ski.T).reshape(n, d_idx, m).permute(2, 0, 1)
  )
  sidx = sidx / norm
  sidx = sidx.repeat(batch_m, 1, 1)
  return sidx

def _fast_kernel_idx(idxu, idxv, k, _idx_part):
  """
  k: callable: A x B x C -> A x B x C
  idxu: M x d_u x d_idx
  idxv: N x d_v x d_idx
  """
  ### idxu MUST be the input mini-batch
  batch_m = 1 # idxu.shape[0]
  # idxu = idxu.mean(dim=0).unsqueeze(0)
  ###
  m, d_u, d_idx = idxu.shape
  n, d_v, _ = idxv.shape
  assert d_idx == idxv.shape[-1]
  # kidxu: M x d_u x d_idx
  # kidxv: N x d_v x d_idx
  kidxu = k(idxu, _idx_part)
  kidxv = k(idxv, _idx_part)
  assert kidxu.shape == idxu.shape
  assert kidxv.shape == idxv.shape
  # kiTi: (M * d_idx) x d_idx(k)
  # kjTj: (N * d_idx) x d_idx(k)
  iTki = torch.bmm(idxu.permute(0, 2, 1), kidxu).reshape(m * d_idx, d_idx)
  jTkj = torch.bmm(idxv.permute(0, 2, 1), kidxv).reshape(n * d_idx, d_idx)
  # iTki_kjTj: M x N x d_idx x d_idx
  iTki_kjTj = (iTki @ jTkj.T).reshape(m, d_idx, n, d_idx).permute(0, 2, 1, 3)
  diag = torch.diagonal(iTki_kjTj, dim1=2, dim2=3)
  ###
  ski = kidxu.sum(dim=1)
  skj = kidxv.sum(dim=1)
  norm = (ski @ skj.T).unsqueeze(-1)
  diag = diag / norm
  ###
  diag = diag.repeat(batch_m, 1, 1)
  return diag

def _kernel_idx(idxu, idxv, k, _idx_part):
  """
  k: callable: A x B x C -> A x B x C
  idxu: M x d_u x d_idx
  idxv: N x d_v x d_idx
  """
  m, d_u, d_idx = idxu.shape
  n, d_v, _ = idxv.shape
  assert d_idx == idxv.shape[-1]
  # kidxu: M x d_u x d_idx
  # kidxv: N x d_v x d_idx
  kidxu = k(idxu, _idx_part)
  kidxv = k(idxv, _idx_part)
  assert kidxu.shape == idxu.shape
  assert kidxv.shape == idxv.shape
  # ski: (M * N) x d_idx
  # skj: (M * N) x d_idx
  # norm: M x N x 1
  ski = kidxu.sum(dim=1)
  skj = kidxv.sum(dim=1)
  norm = (ski @ skj.T).unsqueeze(-1)
  ski = ski.unsqueeze(1).repeat(1, n, 1).reshape(m * n, d_idx, 1)
  skj = skj.unsqueeze(1).repeat(m, 1, 1).reshape(m * n, d_idx, 1)
  # idxu, kidxu: (M * d_u) x d_idx x 1
  # idxv, kidxv: (N * d_v) x d_idx x 1
  idxu = idxu.reshape(m * d_u, d_idx, 1)
  idxv = idxv.reshape(n * d_v, d_idx, 1)
  kidxu = kidxu.reshape(m * d_u, d_idx, 1)
  kidxv = kidxv.reshape(n * d_v, d_idx, 1)
  # sikiT: M x d_idx x d_idx
  # sjkjT: N x d_idx x d_idx
  sikiT = torch.bmm(idxu, kidxu.permute(0, 2, 1))
  sikiT = sikiT.reshape(m, d_u, d_idx, d_idx).sum(dim=1)
  sjkjT = torch.bmm(idxv, kidxv.permute(0, 2, 1))
  sjkjT = sjkjT.reshape(n, d_v, d_idx, d_idx).sum(dim=1)
  del kidxu
  del kidxv
  del idxu
  del idxv
  # sikiT: (M * N) x d_idx x d_idx
  # sjkjT: (M * N) x d_idx x d_idx
  sikiT = sikiT.unsqueeze(1).repeat(1, n, 1, 1).reshape(m * n, d_idx, d_idx)
  sjkjT = sjkjT.unsqueeze(0).repeat(m, 1, 1, 1).reshape(m * n, d_idx, d_idx)
  # diag_sikiT_skjjT: (M * N) x d_idx
  # skjjT = sjkjT.permute(0, 2, 1)
  # diag_sikiT_skjjT = torch.diagonal(torch.bmm(sikiT, skjjT), dim1=1, dim2=2)
  # diag_sikiT_skjjT = diag_sikiT_skjjT.unsqueeze(-1)
  xor_idx = torch.bmm(sikiT, skj) + torch.bmm(sjkjT, ski)
  # xor_idx = torch.bmm(sikiT, skj) + torch.bmm(sjkjT, ski) - diag_sikiT_skjjT
  # xor_idx = diag_sikiT_skjjT
  xor_idx = xor_idx.reshape(m, n, d_idx)
  xor_idx = xor_idx / norm
  return xor_idx

### Classe Tensor Maromba

In [24]:
class MTensor:
  def __init__(
      self,
      values: torch.Tensor,
      indices: torch.Tensor,
      indexer: nn.Module=nn.Identity(),
    ):
    assert values.shape == indices.shape[:-1]
    self.data = values
    self.idx = indices
    self.idx_dim = indices.shape[-1]
    self.indexer = indexer
    self._idx_part = img_dim
    self._eps = 1e-6

  def __getitem__(self, idx):
    return MTensor(self.data[idx], self.idx[idx], self.indexer)

  def __setitem__(self, idx, value):
    self.data[idx] = value.data
    self.idx[idx] = value.idx

  def __delitem__(self, idx):
    del self.data[idx]
    del self.idx[idx]

  def __len__(self):
    return len(self.data)

  @staticmethod
  def cat(mts, dim=0):
    if dim == -1:
      dim = len(mt.data.shape) - 1
    values = [mt.data for mt in mts]
    indices = [mt.idx for mt in mts]
    values = torch.cat(values, dim=dim)
    indices = torch.cat(indices, dim=dim)
    mt = MTensor(values, indices)
    return mt

  @staticmethod
  def unsqueeze(mt, dim=0):
    assert dim != -1
    assert dim < len(mt.idx.shape) - 1
    mt.data = mt.data.unsqueeze(dim)
    mt.idx = mt.idx.unsqueeze(dim)
    return mt

  @staticmethod
  def squeeze(mt, dim=0):
    assert dim != -1
    assert dim < len(mt.idx.shape) - 1
    mt.data = mt.data.squeeze(dim)
    mt.idx = mt.idx.squeeze(dim)
    return mt

  @staticmethod
  def clone(mt):
    return MTensor(mt.data, mt.idx, mt.indexer)

  @staticmethod
  def reshape(mt, shape):
    idx_shape = shape + (mt.idx_dim,)
    nmt = MTensor(
        mt.data.reshape(shape),
        mt.idx.reshape(idx_shape),
        mt.indexer
    )
    return nmt

  @staticmethod
  def permute(mt, perm):
    idx_perm = perm + (-1,)
    nmt = MTensor(
        mt.data.permute(*perm),
        mt.idx.permute(*idx_perm),
        mt.indexer
    )
    return nmt

  def __matmul__(self, b):
    """
    Useful for computing m-product between a batch of inputs (N x ...) and a
    parameter matrix (m x n).

    self.data: pre_shape(self) x in_dim(self)
    self.data.idx: pre_shape(self) x in_dim(self) x d_idx
    b.data: pre_shape(b) x in_dim(b)
    b.idx: pre_shape(b) x in_dim(b) x d_idx

    Returns "mdot"
    mdot.data: pre_shape(self) x pre_shape(b)
    mdot.idx: pre_shape(self) x pre_shape(b) x d_idx
    """
    apre = self.data.shape[:-1]
    bpre = b.data.shape[:-1]
    d_idx = self.idx.shape[-1]
    assert d_idx == b.idx.shape[-1]
    aidx = self.idx.reshape(*((-1,) + self.idx.shape[-2:]))
    bidx = b.idx.reshape(*((-1,) + b.idx.shape[-2:]))
    kernel = _soft_kernel
    ###
    mdot = _nsbmd(
        self.data.reshape(-1, self.data.shape[-1], 1),
        b.data.reshape(-1, b.data.shape[-1], 1),
        aidx,
        bidx,
    )
    ###
    # mdot = _rdot(
    #     self.data.reshape(-1, self.data.shape[-1], 1),
    #     b.data.reshape(-1, b.data.shape[-1], 1),
    #     aidx,
    #     bidx,
    # )
    ###
    mdot = mdot.reshape(apre + bpre)
    # New indices
    # ###
    onesa = torch.ones(self.idx.shape).to(self.idx.device)
    onesb = torch.ones(b.idx.shape).to(b.idx.device)
    # ###
    midx = (
        _nsbmd(aidx, onesb, aidx, bidx)
        + _nsbmd(onesa, bidx, aidx, bidx)
    ) / 2.0
    ###
    # midx = (
    #     _nsbmd(aidx, 1.0 - bidx, aidx, bidx)
    #     + _nsbmd(1.0 - aidx, bidx, aidx, bidx)
    # )
    # midx = _nsbmd(aidx, bidx, aidx, bidx)
    ###
    # midx = (
    #     _rdot(aidx, onesb, aidx, bidx)
    #     + _rdot(onesa, bidx, aidx, bidx)
    # ) / 2.0
    ###
    new_shape = apre + bpre + (d_idx,)
    midx = midx.reshape(new_shape)
    #
    mdot = MTensor(mdot, midx, self.indexer)
    return mdot

  def __mul__(self, b):
    """
    self: N x out_a x in_a (x d_idx)
    b:    N x out_b x in_b (x d_idx)
    """
    n, out_a, in_a = self.data.shape
    assert b.data.shape[0] == n
    _, out_b, in_b = b.data.shape
    d_idx = self.idx.shape[-1]
    assert b.idx.shape[-1] == d_idx
    ### Solução provisória. Calcular o índice com paralelismo ainda não é possível.
    mdots = [MTensor.unsqueeze(self[idx] @ b[idx], dim=0) for idx in range(n)]
    mdots = MTensor.cat(mdots, dim=0)
    return mdots

### Classe do Módulo Treinável

#### Sampling functions

In [16]:
# def idx2d(
#     channels: int,
#     rows: int,
#     cols: int,
#     w: int,
#     h: int,
#     stride: int=2,
#     dilation: int=1,
#     device="cpu"
#   ):
#   idx = []
#   dilh = 1 + dilation * (h - 1)
#   dilw = 1 + dilation * (w - 1)
#   for row in range(0, rows - (dilh - 1), stride):
#     for col in range(0, cols - (dilw - 1), stride):
#       for ch in range(channels):
#         for drow in range(0, dilh, dilation):
#           for dcol in range(0, dilw, dilation):
#             idx.append(
#                 cols * rows * ch
#                 + cols * (row + drow)
#                 + (col + dcol)
#             )
#   idx = torch.tensor(idx).long().to(device)
#   return idx

# def unsort(idxs):
#   ridxs = [0 for _ in idxs]
#   for i, idx in enumerate(idxs):
#     ridxs[idx] = i
#   ridxs = torch.tensor(ridxs).long().to(idxs.device)
#   return ridxs

# def get_perms(tmp_idx):
#   idxs, _idxs = [], []
#   for dim in range(tmp_idx.shape[-1]):
#     ordering = torch.argsort(tmp_idx[:, dim], stable=True)
#     idxs.append(ordering.cpu().detach())
#     _idxs.append(unsort(ordering).cpu().detach())
#   return idxs, _idxs

# def resort(k, src, tgt):
#   assert src == 0 or tgt == 0
#   global idxs, _idxs
#   if tgt == 0:
#     return idxs[src][k]
#   return _idxs[tgt][k]

# def hoods(dims, k0, w, _min=0, _max=None):
#   assert len(dims) == len(w), f"{len(dims)} != {len(w)}"
#   if len(dims) == 0:
#     return [k0] # [k0.item()]
#   _hoods = []
#   global idxs, _idxs
#   _k0d = resort(k0, 0, dims[-1]) #, idxs, _idxs)
#   for _w in range(-(w[-1] // 2), (w[-1] // 2) + (w[-1] % 2)):
#     # k0d = min(_max, max(_min, _k0d + _w))
#     k0d = torch.clip(_k0d + _w, min=_min, max=_max)
#     _hoods += hoods(
#         dims[:-1],
#         resort(
#             k0d,
#             dims[-1], 0,
#             # idxs, _idxs
#         ),
#         w[:-1],
#         # idxs, _idxs,
#         _min, _max
#     )
#   return _hoods

# idxs, _idxs = None, None

# def _idxhood(xidx, ws, stride):
#   """
#   xidx: in_dim x idx_dim
#   """
#   dims = tuple(range(xidx.shape[-1]))
#   global idxs, _idxs
#   idxs, _idxs = get_perms(xidx)
#   pivots = torch.tensor([piv for piv in range(0, len(xidx), stride)]).long()
#   all_hoods = hoods(dims, pivots, ws, 0, len(xidx) - 1)
#   # all_hoods = torch.tensor(all_hoods).long().T.reshape(-1)
#   all_hoods = torch.cat(all_hoods, dim=0).reshape(len(all_hoods), -1).T
#   all_hoods = all_hoods.reshape(-1)
#   # Pdb().set_trace()
#   return all_hoods

from sklearn.neighbors import NearestNeighbors

def idxhood(xidx, ws, stride=None, num_sets=None, sample=False):
  xidx = xidx.reshape(-1, xidx.shape[-1]).cpu().detach().numpy()
  # set desired number of neighbors
  nneigh = int(np.prod(ws))
  neigh = NearestNeighbors(n_neighbors=nneigh)
  neigh.fit(xidx)
  # select indices of k nearest neighbors of the vectors in the input list
  if sample:
    if num_sets is None:
      num_sets = (len(xidx) + (len(xidx) % stride)) // stride
    subidx = np.random.choice(len(xidx), size=num_sets, replace=False)
    all_hoods = neigh.kneighbors(
        xidx[subidx], return_distance=False
    ).reshape(-1)
  else:
    _neigh = NearestNeighbors(n_neighbors=nneigh)
    _ids = np.array([[pos // img_dim, pos % img_dim] for pos in range(len(xidx))])
    # _ids = np.array([[pos] for pos in range(len(xidx))])
    _neigh.fit(_ids)
    if stride is None:
      stride = len(xidx) // num_sets
    all_hoods = _neigh.kneighbors(
        _ids[::stride], return_distance=False
    ).reshape(-1)
  all_hoods = torch.from_numpy(all_hoods).long()
  return all_hoods

#### MModule III

In [ ]:
# from pandas.core.arrays.categorical import Shape

class MModule3(nn.Module):
  def __init__(
      self, n_params=600, idx_dim=3, samples=32, sets=64, device="cpu",
      probe_dim=None,
      ):
    super().__init__()
    self.idx_dim = idx_dim
    self._samples = samples
    self.samples = [int(np.prod(samp)) for samp in samples]
    self.sets = sets
    self.device = device
    self.n_params = n_params
    self.W, self.W_idx, self.MW = self._make_pmt(
        (1, n_params), idx_dim, device
    )
    if probe_dim:
      n_classes = 10
      self._pw, self._pw_idx, self.probe = self._make_pmt(
          (n_classes, probe_dim), idx_dim, device
      )
    self.activation = nn.ELU()

  def _make_pmt(self, shape, idxdim, device):
    _W = nn.Parameter(
        2.0 * torch.rand(shape, device=device) - 1.0
    )
    _W_idx = nn.Parameter(
        2.0 * torch.rand((*shape, idxdim), device=device) - 1.0
    )
    # _W_idx = (
    #     2.0 * torch.rand((*shape, idxdim), device=device) - 1.0
    # )
    _mt = MTensor(_W, _W_idx)
    return _W, _W_idx, _mt

  def forward(self, x: MTensor):
    """
    x.data: N x in_dim
    x.idx: N x in_dim x idx_dim
    """
    n_sets, n_samples = len(self.sets), len(self.samples)
    assert n_sets == n_samples
    assert n_sets > 0
    ### Under experimentation
    n = x.data.shape[0]
    filter_whs = [(samp[1], samp[2], samp[0]) for samp in self._samples[:-1]]
    strides = self.sets[:-1]
    stride = strides[0]
    filter_volume = np.prod(filter_whs[0])
    self.all_pools = [x[:4]]
    idxx = idxhood(x.idx[0], filter_whs[0], strides[0], sample=True) ### FIX
    # pool: N x (num_windows * window_volume)
    pool = x[:, idxx]
    ###
    wl, wr = 0, self.sets[0] * self.samples[0]
    for step in range(n_sets):
      activate = (step < n_sets - 1)
      conv = activate
      mw = MTensor.reshape(
          self.MW[0, wl: wr],
          (self.sets[step], self.samples[step])
      )
      if conv:
        # pool: (N * num_windows) x  sets
        pool = MTensor.reshape(pool, (-1, filter_volume)) @ mw
        pool = MTensor.reshape(pool, (n, -1, self.sets[step]))
      else:
        # pool.data = self.probe(pool.data)
        # pool: N x n_classes
        pool = pool @ self.probe
      nxt_conv = (step + 1 < n_sets - 1)
      if conv:
        # pool: N x num_windows x numw_windows
        self.all_pools.append(pool[:4])
        n, img_area, channels = pool.data.shape
        pool = MTensor.reshape(pool, (n, -1))
        nxt_conv_step = (step + 1) % len(strides)
        stride = strides[nxt_conv_step]
        filter_volume = np.prod(filter_whs[nxt_conv_step])
        if nxt_conv:
          idxx = idxhood(
              pool.idx[0],
              filter_whs[nxt_conv_step],
              strides[nxt_conv_step],
              sample=True,
          ) ### FIX
          pool = pool[:, idxx]
      nxt_step = (step + 1) % n_sets
      next_wr = wr + self.sets[nxt_step] * self.samples[nxt_step]
      wl, wr = wr, next_wr
    return pool

#### MModule IV

In [25]:
def _count_params(config):
  tot_samples = [np.prod(smp) for smp in config["params"]["samples"]]
  n_params = int(np.dot(config["params"]["sets"], tot_samples))
  return n_params

class MModule4(nn.Module):
  def __init__(self, config, idx_dim=3, device="cpu"):
    super().__init__()
    assert len(config["features"]["sets"]) > 0
    assert len(config["params"]["sets"]) > 0
    assert len(config["features"]["sets"]) == len(config["params"]["sets"])
    for key in ["sets", "samples"]:
      assert len(config["features"][key]) == len(config["features"]["sets"])
      assert len(config["params"][key]) == len(config["params"]["sets"])
    self._idx_dim = idx_dim
    self.device = device
    self._config = config
    self._config["params"]["samples"] = [np.prod(smp) for smp in config["params"]["samples"]]
    self._ch0 = config["features"]["pre-set"]
    self._is_conv = config["params"]["is conv"]
    self._feat_samples = config["features"]["samples"]
    self._n_params = _count_params(config)
    self.W, self.W_idx, self.MW = self._make_pmt(
        (self._n_params,), idx_dim, device
    )
    _std = 0.1
    self._ones_idx = nn.Parameter(
        _std * torch.randn((1, 1, idx_dim), device=device)
    )
    self.activation = nn.ELU()
    # self.activation = nn.ReLU()

  def _make_pmt(self, shape, idxdim, device):
    # _W = nn.Parameter(
    #     2.0 * torch.rand(shape, device=device) - 1.0
    # )
    # _W_idx = nn.Parameter(
    #     2.0 * torch.rand((*shape, idxdim), device=device) - 1.0
    # )
    _std = 0.005
    _W = nn.Parameter(
        _std * torch.randn(shape, device=device)
    )
    _W_idx = nn.Parameter(
        _std * torch.randn((*shape, idxdim), device=device)
    )
    # _W_idx = _std * torch.randn((*shape, idxdim), device=device)
    _mt = MTensor(_W, _W_idx)
    return _W, _W_idx, _mt

  def _put_one(self, x: MTensor):
    """
    x.data: N x in_dim
    x.idx: N x in_dim x idx_dim
    """
    n, in_dim, idx_dim = x.idx.shape
    assert x.data.shape == (n, in_dim)
    device = x.data.device
    assert device == x.idx.device
    ones_idx = self._ones_idx.repeat(n, 1, 1)
    dummy_ones = MTensor(
        values=torch.ones(n, 1).to(device),
        # indices=torch.zeros(n, 1, idx_dim).to(device),
        indices=ones_idx,
    )
    x = MTensor.cat([x, dummy_ones], dim=1)
    return x

  def forward(self, x: MTensor):
    """
    x.data: N x in_dim
    x.idx: N x in_dim x idx_dim
    """
    n_layers = len(self._config["params"]["sets"])
    n = x.data.shape[0]
    param_sets = self._config["params"]["sets"]
    param_samples = self._config["params"]["samples"]
    feat_sets = self._config["features"]["sets"]
    feat_samples = self._config["features"]["samples"]
    self.all_pools = []
    pool = x
    pool = MTensor.reshape(pool, (n, self._ch0, -1))
    pool = MTensor.permute(pool, (0, 2, 1))
    wl, wr = 0, param_sets[0] * param_samples[0]
    for step in range(n_layers):
      ###
      if self._is_conv[step]:
        idx_slice = pool.idx[0, :, 0]
      else:
        pool = MTensor.reshape(pool, (n, -1))
        idx_slice = pool.idx[0]
      # pool = MTensor.reshape(pool, (n, -1))
      # idx_slice = pool.idx[0]
      idxx = idxhood(
          idx_slice,
          feat_samples[step],
          num_sets=feat_sets[step],
          sample=True,
      )
      # pool: N x (num_windows * window_volume)
      pool = pool[:, idxx]
      ###
      mw = MTensor.reshape(
          self.MW[wl: wr],
          (param_sets[step], param_samples[step])
      )
      # pool: (N * num_windows) x sets
      # pool = MTensor.reshape(pool, (-1, feat_samples[step])) @ mw
      ### o 1 do bias parece que atrapalha
      # pool = (
      #     self._put_one(MTensor.reshape(
      #         pool, (n * feat_sets[step], -1)
      #     ))
      #     @ mw
      # )
      pool = (
          MTensor.reshape(
              pool, (n * feat_sets[step], -1)
          )
          @ mw
      )
      ###
      # pool = (
      #     MTensor.reshape(pool, (n * feat_sets[step], -1))
      #     @ mw
      # )
      pool = MTensor.reshape(pool, (n, -1))
      # pool: N x (num_windows * sets)
      self.all_pools.append(pool[:4])
      if step < n_layers - 1:
        pool.data = self.activation(pool.data)
        pass
      else:
        break
      ###
      pool = MTensor.reshape(pool, (n, feat_sets[step], param_sets[step]))
      ###
      nxt_step = step + 1
      next_wr = wr + param_sets[nxt_step] * param_samples[nxt_step]
      wl, wr = wr, next_wr
    return pool

### Função de Custo

In [18]:
from sklearn.metrics import accuracy_score

def _check_shapes(y_true, y_pred, true_index, pred_index):
  n, d_out = y_true.shape
  assert y_true.shape[0] == y_pred.shape[0]
  assert true_index.shape[0] == pred_index.shape[0]
  assert true_index.shape[-1] == pred_index.shape[-1]

def _maromba_loss(y_true, y_pred, true_index, pred_index):
  """
  y_true: N x d_out(true)
  y_pred: N x d_out(pred)
  true_index: N x d_out(true) x d_index
  pred_index: N x d_out(pred) x d_index
  """
  _check_shapes(y_true, y_pred, true_index, pred_index)
  # index_match: N x d_out(pred) x d_out(true)
  ###
  pred_index = MTensor._soft_kernel(pred_index, img_dim)
  # pred_index = MTensor._cosine_kernel(pred_index)
  ###
  index_match = torch.bmm(pred_index, true_index.permute(0, 2, 1))
  ### Under experimentation
  # index_match = nn.functional.softmax(index_match, dim=-1)
  ###
  # y_true_match: N x 1 x d_out(pred)
  # y_pred_match: N x 1 x d_out(true)
  y_pred_match = torch.bmm(y_pred.unsqueeze(1), index_match)
  y_true_match = torch.bmm(y_true.unsqueeze(1), index_match.permute(0, 2, 1))
  # huber = nn.HuberLoss()
  # match_loss_lr = huber(y_pred, y_true_match.squeeze(1))
  # match_loss_rl = huber(y_true, y_pred_match.squeeze(1))
  # loss = match_loss_lr + match_loss_rl
  ce = nn.CrossEntropyLoss() # nn.NLLLoss() #
  loss_lr = ce(y_pred_match.squeeze(1), torch.argmax(y_true, dim=-1))
  # loss_rl = ce(y_true_match.squeeze(1), torch.argmax(y_pred, dim=-1))
  loss_rl = ce(y_pred, torch.argmax(y_true_match.squeeze(1), dim=-1))
  loss = loss_lr + loss_rl
  return loss

def _pool2category(y_true, y_pred, true_index, pred_index):
  _check_shapes(y_true, y_pred, true_index, pred_index)
  # index_match: N x d_out(pred) x d_out(true)
  index_match = torch.bmm(pred_index, true_index.permute(0, 2, 1))
  y_pred_match = torch.bmm(y_pred.unsqueeze(1), index_match)
  y_pred_match = torch.argmax(y_pred_match.squeeze(1), dim=-1).tolist()
  return y_pred_match

def _maromba_accuracy(y_true, y_pred, true_index, pred_index):
  ###
  # pred_index = MTensor._cosine_kernel(pred_index)
  pred_index = MTensor._soft_kernel(pred_index, img_dim)
  ###
  y_pred_match = _pool2category(y_true, y_pred, true_index, pred_index)
  y_true = torch.argmax(y_true, dim=-1).tolist()
  acc = accuracy_score(y_true, y_pred_match)
  return acc

def maromba_accuracy(y_true, y_pred):
  return _maromba_accuracy(y_true.data, y_pred.data, y_true.idx, y_pred.idx)

def maromba_loss(y_true, y_pred):
  return _maromba_loss(y_true.data, y_pred.data, y_true.idx, y_pred.idx)

def regular_accuracy(y_true, y_pred):
  y_true = torch.argmax(y_true.data, dim=-1).tolist()
  y_pred = torch.argmax(y_pred.data, dim=-1).tolist()
  acc = accuracy_score(y_true, y_pred)
  return acc

def regular_loss(y_true, y_pred):
  y_true = y_true.data
  # y_pred = 10.0 * y_pred.data ### WHY 10x?
  y_pred = y_pred.data ### WHY 10x?
  ce = nn.CrossEntropyLoss()
  loss = ce(y_pred, torch.argmax(y_true, dim=-1))
  return loss

maromba_loss = regular_loss
maromba_accuracy = regular_accuracy

### Inicialização

In [19]:
num_classes = 10
rows, cols = img_dim, img_dim
hidden_dim = 1 * img_dim
clf_dim = (1 + (num_classes - 1) // img_dim) * img_dim
idx_dim = 3 # rows + cols + hidden_dim + clf_dim # 3

# template_x_idx = _cat2d(rows, cols, d=idx_dim)
template_x_idx = cartesian_idx(rows, cols, chs=channels, offset=1, d=idx_dim) ### offset=0
template_x_idx = template_x_idx.unsqueeze(0).float().to(device)
# template_y_idx = torch.eye(idx_dim)[-num_classes:]
template_y_idx = torch.eye(num_classes)[:, -idx_dim:]
template_y_idx = template_y_idx.float().unsqueeze(0).to(device)

def prepare_input(x, y, device="cpu"):
  n = x.shape[0]
  x_idx = template_x_idx.repeat(n, 1, 1)
  yoh = torch.zeros(n, num_classes)
  yoh[range(n), y] = 1.0
  yoh = yoh.to(device)
  y_idx = template_y_idx.repeat(n, 1, 1)
  x = MTensor(x, x_idx)
  y = MTensor(yoh, y_idx)
  return x, y

### Treino (a)

In [ ]:
# tidx = model.W_idx[0, :18].cpu().detach().numpy()
# plot_df = pd.DataFrame({"x": tidx[:, 0], "y": tidx[:, 1], "z": tidx[:, 2]})
# fig = px.scatter_3d(plot_df, x="x", y="y", z="z", color=None); fig.show();

In [ ]:
# tidx = aidx.reshape(32, -1, 9, 3)[0, 100].cpu().detach().numpy(); tidx = tidx.reshape(-1, 3)
# tidx = aidx.reshape(32, -1, 9, 2)[0].cpu().detach().numpy(); tidx = tidx.reshape(-1, 2)

# plot_df = pd.DataFrame({"x": tidx[:, 0], "y": tidx[:, 1], "z": tidx[:, 1] * 0.0})
# fig = px.scatter_3d(plot_df, x="x", y="y", z="z", color=None); fig.show();
# midx.reshape(32, -1, 3)[0, 100:105].cpu().detach().numpy()
torch.cuda.empty_cache()

In [ ]:
df_train

In [ ]:
imgs = [img.data.cpu().detach() for img in model.all_pools]
idxs = [img.idx.cpu().detach() for img in model.all_pools]

In [ ]:
[img.shape for img in imgs]

In [ ]:
shapes = [
    (img_dim, img_dim, 1),
    ((img_dim - 0) // 1, (img_dim - 0) // 1, 1),
    # ((img_dim - 0) // 1, (img_dim - 0) // 1, 4),
    # ((img_dim - 0) // 1, (img_dim - 0) // 1, 8),
    # (((img_dim - 2) // 2 - 2) // 1, ((img_dim - 2) // 2 - 2) // 1, 4),
    # (((img_dim - 4) // 1 - 4) // 2, ((img_dim - 4) // 1 - 4) // 2, 8),
    # ((img_dim - 4) // 2 - 2, (img_dim - 4) // 2 - 2, 4),
    # ((img_dim - 4) // 2 - 4, (img_dim - 4) // 2 - 4, 8),
]
# display.clear_output(wait=True)
plt.close()
for idx, img in enumerate(imgs[:len(shapes)]):
  img = img[0]
  print(img.shape)
  img = img.reshape(*shapes[idx])
  rows, cols = 1, shapes[idx][2]
  fig, ax = plt.subplots(rows, cols, figsize=(min(18, 3 * cols), 3))
  for ch in range(cols):
    img_ = img[:, :, ch].numpy()
    if cols > 1:
      ax[ch].imshow(img_)
    else:
      ax.imshow(img_)
  plt.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def scatter3d(x, y, z):
  plot_df = pd.DataFrame({"x": x, "y": y, "z": z})
  fig = px.scatter_3d(plot_df, x="x", y="y", z="z", color=None)
  return fig

# plt.clf(); plt.cla()
# plt.close()
for i, idx in enumerate(idxs[:len(shapes)]):
  idx = idx[0]
  print(idx.shape)
  idx = idx.reshape(-1, shapes[i][2], idx_dim)
  rows, cols = 1 + (idx.shape[1] - 1) // 2, 2
  # fig = plt.figure(figsize=(min(18, 3 * cols), 3))
  fig = make_subplots(
    rows=rows, cols=cols,
    specs=[[{"type": "scene"} for _ in range(cols)] for _ in range(rows)],
    # row_heights=[10 for _ in range(rows)],
    vertical_spacing=0.05
  )
  fig.update_layout(
    height=600 * rows,
    width=600 * cols
)
  for ch in range(idx.shape[1]):
    idx_ = idx[:, ch].numpy()
    # ax = fig.add_subplot(rows, cols, ch + 1, projection="3d")
    # ax.scatter(idx_[::4, 0], idx_[::4, 1], idx_[::4, 2], marker="+")
    # fig = scatter3d(idx_[::4, 0], idx_[::4, 1], idx_[::4, 2])
    row, col = (ch // cols) + 1, (ch % cols) + 1
    fig.add_trace(
        go.Scatter3d(
            x=idx_[::, 0],
            y=idx_[::, 1],
            z=idx_[::, 2],
            # z=idx_[::, 1] * 0.0,
            # color=None,
            # colorscale="Viridis",
            # showscale=False
        ),
        row=row,
        col=col,
    )
  fig.show()
  # plt.show()

In [ ]:
# model.MW.idx[0, 0]
threshold = rows * cols * hidden_dim
param_id = 0
layer_idx = model.MW.idx[:, param_id: param_id + 100] # threshold
n_parts = idx_dim // img_dim
scaled_idx = (
    MTensor
    ._soft_kernel(layer_idx, img_dim)[0]
    .reshape(-1, n_parts, img_dim)
) * (n_parts ** 0.5)
idx_att = torch.argmax(
    scaled_idx,
    dim=-1
)[:, :2]
idx_att = idx_att.cpu().detach().numpy()
grid = np.zeros((rows, cols))
# for pos in range(len(idx_att)):
#   idxx, idxy = idx_att[pos]
#   idxx, idxy = int(idxx), int(idxy)
#   # grid[int(idxx), int(idxy)] += 1
#   grid[idxx, idxy] += scaled_idx[pos, 0, idxx] * scaled_idx[pos, 1, idxy]
for idxx in range(rows):
  for idxy in range(cols):
    grid[idxx, idxy] = (scaled_idx[:, 0, idxx] * scaled_idx[:, 1, idxy]).sum()
grid = grid / grid.max()

# plt.imshow(grid)

fig, ax = plt.subplots()
ax.matshow(grid, cmap="seismic")

for (i, j), z in np.ndenumerate(grid):
    ax.text(j, i, "{:0.2f}".format(z), ha="center", va="center")

plt.show()

In [ ]:
MTensor._soft_kernel(y_pred.idx, img_dim)[0]

### Treino (b)

In [33]:
hidden_dim = 10
start_mode = False
valid_mode = False

# TODO: Visualize conv layer output
###
# config = {
#     "features": {
#         "pre-set": channels,
#         "sets":    [1024,    1024,    1,],
#         "samples": [25,      1,       1024,],
#     },
#     "params": {
#         "sets":    [4,       1,       num_classes,],
#         "samples": [(76, 1), (1, 5),  (1025, 1),],
#         "is conv": [True,    True,    False,]
#     },
# }
###
# config = {
#     "features": {
#         "pre-set": channels,
#         "sets":    [784,     784,     784,     1,],
#         "samples": [9,       9,       1,       784,],
#     },
#     "params": {
#         "sets":    [8,       16,      1,       num_classes,],
#         "samples": [(3, 1),  (9, 1),  (1, 4),  (28, 1),],
#         "is conv": [True,    True,    True,    False,]
#     },
# }
###
config = {
    "features": {
        "pre-set": channels,
        "sets":    [784,   784,   1,],
        "samples": [9,     18,    2*784,],
    },
    "params": {
        "sets":    [2,     2,     num_classes,],
        "samples": [9,     18,    100,],
        "is conv": [False, False, False]
    },
}

tot_samples = [np.prod(smp) for smp in config["params"]["samples"]]
conv_params = int(np.dot(config["params"]["sets"][:-1], tot_samples[:-1]))
n_params = int(np.dot(config["params"]["sets"], tot_samples))

if start_mode:
  model = MModule4(
      config=config,
      idx_dim=idx_dim,
      device=device,
  )
  optimizer = Adam(model.parameters(), lr=1e-2)
  # optimizer = SGD(model.parameters(), lr=1e-3, momentum=0.9)
  widx0 = model.W_idx[:, :conv_params].clone().detach()
  train_log = {
      "train loss": [],
      "eval loss": [],
      "acc": [],
      "set": [],
      "epoch": [],
  }
  epoch = 0

num_epochs = 720 * 4
epoch_len = 60

while epoch < num_epochs:
  epoch += 1
  ###
  # widx_diff = (model.W_idx[:, :conv_params] - widx0)
  # print(widx_diff.abs().mean().item(), widx_diff.min().item(), widx_diff.max().item())
  ###
  model.train()
  train_iter = iter(train_data_loader)
  for _ in range(epoch_len):
    x, y = next(train_iter)
    x = x.to(device)
    y = y.to(device)
    x, y = prepare_input(x, y, device=device)
    y_pred = model.forward(x)
    optimizer.zero_grad()
    loss = maromba_loss(y, y_pred)
    loss.backward()
    optimizer.step()
    train_log["train loss"].append(loss.item())
    train_log["eval loss"].append(np.nan)
    train_log["acc"].append(np.nan)
    train_log["set"].append("train")
    train_log["epoch"].append(epoch)
  if valid_mode:
    model.eval()
    with torch.no_grad():
      for x, y in iter(test_data_loader):
        x = x.to(device)
        y = y.to(device)
        x, y = prepare_input(x, y, device=device)
        y_pred = model.forward(x)
        loss = maromba_loss(y, y_pred)
        acc = maromba_accuracy(y, y_pred)
        train_log["eval loss"].append(loss.item())
        train_log["train loss"].append(np.nan)
        train_log["acc"].append(acc.item())
        train_log["set"].append("eval")
        train_log["epoch"].append(epoch)
    metric_cols = ["eval loss", "acc"]
    set_val = "eval"
  else:
    metric_cols = ["train loss",]
    set_val = "train"
  group_cols = ["epoch"] + metric_cols
  df_train = pd.DataFrame(train_log)
  df_train = df_train[df_train["set"] == set_val]
  display.clear_output(wait=True)
  (
    df_train[group_cols]
    .groupby("epoch")
    .agg(lambda x: x.median(skipna=True))
    .reset_index()
    .sort_values("epoch", ascending=True)
    .tail(30)[metric_cols]
    .plot(figsize=(16, 3), grid=True)
  )
  plt.show()

> <ipython-input-32-48e3218ca5c6>(28)_nsbmd()
     26   """
     27   Pdb().set_trace()
---> 28   m, d_u, d_idx  = idxu.shape
     29   n, d_v, d_idxv = idxv.shape
     30   d_val = u.shape[-1]

ipdb> c
> <ipython-input-32-48e3218ca5c6>(28)_nsbmd()
     26   """
     27   Pdb().set_trace()
---> 28   m, d_u, d_idx  = idxu.shape
     29   n, d_v, d_idxv = idxv.shape
     30   d_val = u.shape[-1]

ipdb> c
> <ipython-input-32-48e3218ca5c6>(28)_nsbmd()
     26   """
     27   Pdb().set_trace()
---> 28   m, d_u, d_idx  = idxu.shape
     29   n, d_v, d_idxv = idxv.shape
     30   d_val = u.shape[-1]

ipdb> idxu.shape
torch.Size([25088, 9, 3])
ipdb> c
> <ipython-input-32-48e3218ca5c6>(28)_nsbmd()
     26   """
     27   Pdb().set_trace()
---> 28   m, d_u, d_idx  = idxu.shape
     29   n, d_v, d_idxv = idxv.shape
     30   d_val = u.shape[-1]

ipdb> idxu.shape
torch.Size([25088, 18, 3])
ipdb> tidx = idxu.reshape(32, -1, 18, 3)[0, 0].cpu().detach().numpy()
ipdb> plot_df = pd.DataFrame({"x": tidx

ipdb> tidx = idxu.reshape(32, -1, 18, 3)[0, 30].cpu().detach().numpy()
ipdb> plot_df = pd.DataFrame({"x": tidx[:, 0], "y": tidx[:, 1], "z": tidx[:, 2]})
ipdb> fig = px.scatter_3d(plot_df, x="x", y="y", z="z", color=None); fig.show();


ipdb> tidx = idxu.reshape(32, -1, 18, 3)[0, -1].cpu().detach().numpy()
ipdb> plot_df = pd.DataFrame({"x": tidx[:, 0], "y": tidx[:, 1], "z": tidx[:, 2]})
ipdb> fig = px.scatter_3d(plot_df, x="x", y="y", z="z", color=None); fig.show();


ipdb> tidx = idxu.reshape(32, -1, 3)[0].cpu().detach().numpy()
ipdb> plot_df = pd.DataFrame({"x": tidx[:, 0], "y": tidx[:, 1], "z": tidx[:, 2]})
ipdb> fig = px.scatter_3d(plot_df, x="x", y="y", z="z", color=None); fig.show();


ipdb> q


In [ ]:
# tidx = idxu.reshape(32, -1, 3)[0].cpu().detach().numpy()
# tidx = idxu.reshape(32, -1, 18, 3)[0, 0].cpu().detach().numpy()
# tidx = idxv.reshape(-1, 3).cpu().detach().numpy()
## tidx = model.W_idx[0, :18].cpu().detach().numpy()
# plot_df = pd.DataFrame({"x": tidx[:, 0], "y": tidx[:, 1], "z": tidx[:, 2]})
# fig = px.scatter_3d(plot_df, x="x", y="y", z="z", color=None); fig.show();

### Visualização dos índices dos parâmetros

In [ ]:
soft_W_idx = scaled_idx # MTensor._soft_kernel(model.MW.idx, img_dim)
threshold = 100 # rows * cols * hidden_dim
# First layer
soft_W_idx = soft_W_idx[:, :threshold].reshape(1, -1, idx_dim)
# Last layer
# soft_W_idx = soft_W_idx[:, threshold:].reshape(1, -1, idx_dim)
soft_W_idx = soft_W_idx.cpu().detach().numpy()[0]

In [ ]:
from sklearn.manifold import TSNE

sample_idx = np.random.choice(
    len(soft_W_idx),
    min(len(soft_W_idx), 10000),
    replace=False
)

W_idx_tsne = TSNE(
    n_components=2,
    perplexity=10,
).fit_transform(soft_W_idx[sample_idx])

In [ ]:
plot_df = pd.DataFrame(
    {
        "W_idx x tsne": W_idx_tsne[:, 0],
        "W_idx y tsne": W_idx_tsne[:, 1],
    }
)

plot_df.plot.scatter(
    x="W_idx x tsne",
    y="W_idx y tsne",
    figsize=(24, 4),
    grid=True
)